In [132]:
import pyodbc
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = '10.0.1.40' 
database = 'Ntier_EPIC' 
username = 'Sisenseuser' 
password = '!w@ntd@t@#123'  
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [133]:
# pd.set_option('display.max_rows',28)
# pd.set_option('display.max_columns',28)

In [134]:
query1 = """
Select T1.IMREDEM_CODE,T1.RESULT_ABNORMAL, T3.Patient_Id, T1.RESULT_NAME, T1.RESULT_VALUE, T1.RESULT_UNITS,
T1.RESULT_NORMAL_RANGE,(convert(date,T1.RESULT_COMPLETIONDATE)) AS RESULT_COMPLETIONDATE 
FROM EMR.HPSITE.LAB_RESULT AS T1 
LEFT JOIN EMR.[HPSITE].[DEMOGRAPHICS] AS T2 ON T1.IMREDEM_CODE=T2.IMREDEM_CODE
LEFT JOIN vwGenPatInfo T3 ON T2.DEM_EXTERNALID=T3.Patient_Number
where T1.RESULT_VALUE IS NOT NULL AND t3.Patient_id='189791' and (T1.RESULT_NAME like '%A1c%')
order by RESULT_COMPLETIONDATE desc; /*174588,75614,85622,144681*/ 
"""
Labresult_data = pd.read_sql(query1, cnxn)
Labresult_data

,IMREDEM_CODE,RESULT_ABNORMAL,Patient_Id,RESULT_NAME,RESULT_VALUE,RESULT_UNITS,RESULT_NORMAL_RANGE,RESULT_COMPLETIONDATE
0,126155,1,189791,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,9.5,%,3.5 - 5.6,2022-05-09
1,126155,1,189791,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,9.5,%,3.5 - 6.0,2021-11-17
2,126155,1,189791,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,13.7,%,3.5 - 6.0,2021-08-16
3,126155,1,189791,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,14.8,%,3.5 - 6.0,2021-05-19


In [135]:
query2 = """Select [Patient_ID],[Service_Date_From],[Procedure_Code],[Primary_Diagnosis_Code]
                            from [vwGenSvcInfo] where Primary_Diagnosis_Code
                            between 'E08' and 'E13' and Patient_ID='189791'
                          order by [Service_Date_From] desc"""
Patient_data = pd.read_sql(query2, cnxn)
Patient_data

,Patient_ID,Service_Date_From,Procedure_Code,Primary_Diagnosis_Code
0,189791,2022-06-15,99214,E11.65
1,189791,2022-06-15,3077F,E11.65
2,189791,2022-06-15,3080F,E11.65
3,189791,2022-06-15,3008F,E11.65
4,189791,2022-05-09,99214,E11.65
5,189791,2022-05-09,36416,E11.65
6,189791,2022-05-09,83036,E11.65
7,189791,2022-04-27,99214,E11.65
8,189791,2022-04-27,3077F,E11.65
9,189791,2022-04-27,G8753,E11.65


In [136]:
def Preprocessing_step_1(Patient_data,Labresult_data):
    import datetime
    formatter='%Y-%m-%d'
    Labresult_data["RESULT_COMPLETIONDATE"] = pd.to_datetime(Labresult_data["RESULT_COMPLETIONDATE"])
    Patient_data['Service_Date_From_trial']=pd.to_datetime(Patient_data['Service_Date_From'])
    # date = Patient_data['Service_Date_From_trial']
    Patient_data['Min']=Patient_data['Service_Date_From_trial']
    remove = lambda x: Patient_data[x].duplicated(keep='last')  
    Patient_data.loc[remove('Service_Date_From_trial'), 'Service_Date_From_trial'] = np.nan
    p=Patient_data['Service_Date_From_trial'][:-1]
    p.index = p.index+1
    Patient_data['Max'] = pd.concat([pd.Series(['']),p]) 
    Patient_data['Min']=pd.to_datetime(Patient_data['Min'])
    Patient_data['Max']=pd.to_datetime(Patient_data['Max'])
    m = Patient_data['Max'].duplicated()
    Patient_data['Max'] = Patient_data['Max'].mask(m).ffill()
    Patient_data.drop('Service_Date_From_trial',axis=1,inplace=True)
    Patient_data["Service_Date_From"] = pd.to_datetime(Patient_data["Service_Date_From"])
    Patient_data["Service_Date_From"] =Patient_data["Service_Date_From"].astype(str)
    min_date,max_date=Patient_data["Service_Date_From"].min(),Patient_data["Service_Date_From"].max()
    lists_of_labcomp_data = Labresult_data["RESULT_COMPLETIONDATE"].unique()
    lists_of_labcomp_data= [pd.to_datetime(str(i)) for i in lists_of_labcomp_data]
    lists_of_labcomp_data.sort()
    date_range = pd.date_range(min_date,max_date)
    date_range = [pd.to_datetime(str(i)) for i in date_range] #parser.parse(str(i)).strftime('%Y-%m-%d')
    results = list(set(date_range).intersection(set(lists_of_labcomp_data)))
    results.sort(reverse=True)
    result_list = results
    resultant_df = pd.DataFrame()
    for i in range(len(result_list)):
        res = Labresult_data.loc[Labresult_data["RESULT_COMPLETIONDATE"] == result_list[i]]
        resultant_df = resultant_df.append(res)   
    return Patient_data,Labresult_data,resultant_df 
   

In [137]:
Patient_data,Labresult_data,resultant_df=Preprocessing_step_1(Patient_data,Labresult_data)

In [138]:
Patient_data
#Labresult_data
#resultant_df

,Patient_ID,Service_Date_From,Procedure_Code,Primary_Diagnosis_Code,Min,Max
0,189791,2022-06-15,99214,E11.65,2022-06-15,NaT
1,189791,2022-06-15,3077F,E11.65,2022-06-15,NaT
2,189791,2022-06-15,3080F,E11.65,2022-06-15,NaT
3,189791,2022-06-15,3008F,E11.65,2022-06-15,NaT
4,189791,2022-05-09,99214,E11.65,2022-05-09,2022-06-15
5,189791,2022-05-09,36416,E11.65,2022-05-09,2022-06-15
6,189791,2022-05-09,83036,E11.65,2022-05-09,2022-06-15
7,189791,2022-04-27,99214,E11.65,2022-04-27,2022-05-09
8,189791,2022-04-27,3077F,E11.65,2022-04-27,2022-05-09
9,189791,2022-04-27,G8753,E11.65,2022-04-27,2022-05-09


In [139]:
def Preprocessing_step_2(Patient_data,resultant_df):
    from datetime import date 
    df_li = []
    today = date.today()
    str(today)
    Patient_data['LabResult'] = None
    pd.to_datetime(1646092800000000000)
    lab_result_dates = list(set(resultant_df['RESULT_COMPLETIONDATE'].values.tolist()))
    lab_result_dates_new = [pd.to_datetime(l) for l in lab_result_dates]
    lab_result_dates_new.sort(reverse=True)
    Lab_result_list1 =lab_result_dates_new
  

    for index,rows in Patient_data.iterrows():
        status = 0
        if rows['Procedure_Code'] == "83036":
            min_date = rows['Min']
            max_date = rows['Max']
            if str(max_date) == "NaT":
                max_date = min_date
                now =  date.today()
                rows['Max'] = now

            for each_result_date in Lab_result_list1:
                each_result_date = pd.to_datetime(each_result_date)
                if each_result_date >= min_date and each_result_date <= max_date:
                    rows['LabResult'] = each_result_date
                    df_li.append(rows)
                    df_li.append(rows.copy())
                    status=1
        if status==0:
            df_li.append(rows)
    Patient_data_new = pd.DataFrame(df_li)
    Patient_data_new['RESULT_COMPLETIONDATE'] = Patient_data_new['LabResult'].apply(lambda x: x.strftime("%Y-%m-%d") if str(x)!= "NaT" else "NA")
    resultant_df['RESULT_COMPLETIONDATE'] = resultant_df['RESULT_COMPLETIONDATE'].apply(lambda x: x.strftime("%Y-%m-%d") if str(x)!= "NaT" else "NA")
    patient_df_lab_record_join = Patient_data_new.merge(resultant_df,on="RESULT_COMPLETIONDATE",how="left")
    return patient_df_lab_record_join,resultant_df
    #patient_df_lab_record_join=patient_df_lab_record_join.drop(['LabResult','IMREDEM_CODE','Patient_Id','RESULT_UNITS'],axis=1)
    #patient_df_lab_record_join.drop_duplicates(subset=['Patient_ID','Service_Date_From','Procedure_Code','Primary_Diagnosis_Code','Min','Max','RESULT_COMPLETIONDATE','RESULT_ABNORMAL','RESULT_NAME','RESULT_VALUE','RESULT_NORMAL_RANGE'],inplace=True)
   

In [140]:
patient_df_lab_record_join,resultant_df=Preprocessing_step_2(Patient_data,resultant_df)
patient_df_lab_record_join

,Patient_ID,Service_Date_From,Procedure_Code,Primary_Diagnosis_Code,Min,Max,LabResult,RESULT_COMPLETIONDATE,IMREDEM_CODE,RESULT_ABNORMAL,Patient_Id,RESULT_NAME,RESULT_VALUE,RESULT_UNITS,RESULT_NORMAL_RANGE
0,189791,2022-06-15,99214,E11.65,2022-06-15,NaT,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,189791,2022-06-15,3077F,E11.65,2022-06-15,NaT,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,189791,2022-06-15,3080F,E11.65,2022-06-15,NaT,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,189791,2022-06-15,3008F,E11.65,2022-06-15,NaT,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,189791,2022-05-09,99214,E11.65,2022-05-09,2022-06-15,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,189791,2022-05-09,36416,E11.65,2022-05-09,2022-06-15,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,189791,2022-05-09,83036,E11.65,2022-05-09,2022-06-15,2022-05-09,2022-05-09,126155.0,1.0,189791.0,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,9.5,%,3.5 - 5.6
7,189791,2022-05-09,83036,E11.65,2022-05-09,2022-06-15,2022-05-09,2022-05-09,126155.0,1.0,189791.0,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,9.5,%,3.5 - 5.6
8,189791,2022-04-27,99214,E11.65,2022-04-27,2022-05-09,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,189791,2022-04-27,3077F,E11.65,2022-04-27,2022-05-09,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
def getderiveddata(patient_df_lab_record_join,app_date):
    import datetime
    import datetime 
    from datetime import datetime, time, date
    #app_date="2022-07-15"
    formatter = '%Y-%m-%d'
    app_date = datetime.strptime(app_date, formatter)
    patient_df_lab_record_join["Next_Appt_date"] = app_date
    patient_df_lab_record_join["Service_Date_From"]=pd.to_datetime(patient_df_lab_record_join["Service_Date_From"])
    #actual_data["Service_Date_To"]=pd.to_datetime(actual_data["Service_Date_To"])
    
    patient_df_lab_record_join["Service_Date_max"] = patient_df_lab_record_join.groupby("Patient_ID")["Service_Date_From"].transform(max)
    #patient_df_lab_record_join["new_date"]=pd.to_datetime(patient_df_lab_record_join["new_date"],errors='coerce')
    #actual_data["last_visit_day"] = (actual_data["new_date"] - actual_data["Service_Date_max"])
    #actual_data["last_visit_day"] = actual_data["last_visit_day"].apply(lambda x: int(str(x).split(" ")[0]))
    patient_df_lab_record_join["Diff_in_days"] = (patient_df_lab_record_join["Next_Appt_date"] - patient_df_lab_record_join["Service_Date_max"])
    patient_df_lab_record_join["Diff_in_days"] = patient_df_lab_record_join["Diff_in_days"].apply(lambda x: int(str(x).split(" ")[0]))
    patient_df_lab_record_join["Patient_Practice"] = patient_df_lab_record_join.loc[:, "Diff_in_days"].apply(lambda x: "Adhoc" if x >= 90 else "Regular")
    patient_df_lab_record_join=patient_df_lab_record_join[['Patient_ID','Service_Date_From','Procedure_Code','Primary_Diagnosis_Code','Next_Appt_date','Service_Date_max','Diff_in_days','Patient_Practice','RESULT_COMPLETIONDATE','RESULT_ABNORMAL','RESULT_NAME','RESULT_VALUE','RESULT_NORMAL_RANGE']]
    return patient_df_lab_record_join

In [142]:
patient_df_lab_record_join=getderiveddata(patient_df_lab_record_join,'2022-07-14') 
patient_df_lab_record_join

,Patient_ID,Service_Date_From,Procedure_Code,Primary_Diagnosis_Code,Next_Appt_date,Service_Date_max,Diff_in_days,Patient_Practice,RESULT_COMPLETIONDATE,RESULT_ABNORMAL,RESULT_NAME,RESULT_VALUE,RESULT_NORMAL_RANGE
0,189791,2022-06-15,99214,E11.65,2022-07-14,2022-06-15,29,Regular,NA,NaN,NaN,NaN,NaN
1,189791,2022-06-15,3077F,E11.65,2022-07-14,2022-06-15,29,Regular,NA,NaN,NaN,NaN,NaN
2,189791,2022-06-15,3080F,E11.65,2022-07-14,2022-06-15,29,Regular,NA,NaN,NaN,NaN,NaN
3,189791,2022-06-15,3008F,E11.65,2022-07-14,2022-06-15,29,Regular,NA,NaN,NaN,NaN,NaN
4,189791,2022-05-09,99214,E11.65,2022-07-14,2022-06-15,29,Regular,NA,NaN,NaN,NaN,NaN
5,189791,2022-05-09,36416,E11.65,2022-07-14,2022-06-15,29,Regular,NA,NaN,NaN,NaN,NaN
6,189791,2022-05-09,83036,E11.65,2022-07-14,2022-06-15,29,Regular,2022-05-09,1.0,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,9.5,3.5 - 5.6
7,189791,2022-05-09,83036,E11.65,2022-07-14,2022-06-15,29,Regular,2022-05-09,1.0,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,9.5,3.5 - 5.6
8,189791,2022-04-27,99214,E11.65,2022-07-14,2022-06-15,29,Regular,NA,NaN,NaN,NaN,NaN
9,189791,2022-04-27,3077F,E11.65,2022-07-14,2022-06-15,29,Regular,NA,NaN,NaN,NaN,NaN


In [143]:
rules_data=pd.read_csv(r"E:/OneDrive - Quadratic Insights Pvt Ltd/Desktop/Epic_Rules.csv")
rules_data

,Diabetic_Patient,Diagnosis_Code,Procedures_Description,Procedure_Code,Freq_Threshold_In_Days
0,Diabetic,E08-E13,HBA1C,83036,90
1,Diabetic,E08-E13,Urinalysis,81002,180
2,Diabetic,E08-E13,Microalbumin w/ Creatinine,82044,180
3,Diabetic,E08-E13,Microalbumin w/ Creatinine,82570,180
4,Diabetic,E08-E13,CMP (comprehensive metabolic panel),80053,90
5,Diabetic,E08-E13,Lipid,80061,120
6,Diabetic,E08-E13,TSH,84443,180
7,Diabetic,E08-E13,Foot exam,2028F,365
8,Diabetic,E08-E13,RETINAL EYE EXAM,92250,365
9,Diabetic,E08-E13,DILATED EYE EXAMINATION PERFORMED,2022F,365


In [144]:
#aic_Procedure=list(patient_df_lab_record_join[patient_df_lab_record_join['Procedure_Code']=='83036']['RESULT_VALUE'])[0]
#aic_Procedure

In [145]:
def recommendation(one_patient_record, rules_data):
    recommendation_procedure_list = {}
    aic_procedure_code = '83036'
    aic_procedure_code_value=list(patient_df_lab_record_join[patient_df_lab_record_join['Procedure_Code']=='83036']['RESULT_VALUE'])[0]
    unique_procedure_code = rules_data["Procedure_Code"].value_counts().keys().tolist()

    for row in range(len(one_patient_record)):
        procedure_code_list = []
        diff_in_days_list = []
        result_value = []

        procedure_code_list.append(one_patient_record.loc[row, "Procedure_Code"])
        diff_in_days_list.append(one_patient_record.loc[row, "Diff_in_days"])
        result_value.append(one_patient_record.loc[row, "RESULT_VALUE"])

        for record in range(len(procedure_code_list)):
            procedure_code = procedure_code_list[record]
            diff_in_days = diff_in_days_list[record]

            if procedure_code in unique_procedure_code:
                res = rules_data.loc[rules_data["Procedure_Code"] == procedure_code]
                rules_procedure_code = res["Procedure_Code"].values[0]
                rules_diff_in_days = res["Freq_Threshold_In_Days"].values[0]

                if rules_procedure_code == procedure_code and int(rules_diff_in_days) <= (diff_in_days):
                    recommendation_procedure_list[procedure_code] = "1"
                else:
                    recommendation_procedure_list[procedure_code] = "0"
                    
                if procedure_code == aic_procedure_code:
                    if (rules_diff_in_days) >= (diff_in_days) and aic_procedure_code_value > '7.0':
                        recommendation_procedure_list[procedure_code] = "1"
                    elif (rules_diff_in_days) >= (diff_in_days) and aic_procedure_code_value > '8.0':
                        recommendation_procedure_list[procedure_code] = "1"
                    elif (rules_diff_in_days) >= (diff_in_days) and aic_procedure_code_value > '9.0':
                        recommendation_procedure_list[procedure_code] = "1"
                    else:
                        recommendation_procedure_list[procedure_code] = "0"

            else:
                for procedure in unique_procedure_code:
                    if procedure not in list(recommendation_procedure_list.keys()):
                        recommendation_procedure_list[procedure] = "1"


    return recommendation_procedure_list

In [146]:
recommendation(patient_df_lab_record_join, rules_data)

{'83036': '1',
 '81002': '0',
 '82044': '0',
 '82570': '0',
 '80053': '0',
 '80061': '0',
 '84443': '0',
 '2028F': '0',
 '92250': '1',
 '2022F': '1',
 '2026F': '0',
 '93922': '0'}

In [ ]:
# recommendation_procedure_list = {}
#             aic_procedure_code = '3046f'

#             unique_procedure_code = rules_data["Procedure_Code"].value_counts().keys().tolist()

#             for row in range(len(one_patient_record)):
#                 procedure_code_list = []
#                 diff_in_days_list = []
#                 result_value = []

#                 procedure_code_list.append(one_patient_record.loc[row, "Procedure_Code"])
#                 diff_in_days_list.append(one_patient_record.loc[row, "Diff_in_days"])
#                 #result_value.append(one_patient_record.loc[row, "RESULT_VALUE"])

#                 for record in range(len(procedure_code_list)):
#                     procedure_code = procedure_code_list[record]
#                     diff_in_days = diff_in_days_list[record]

#                     if procedure_code in unique_procedure_code:
#                         res = rules_data.loc[rules_data["Procedure_Code"] == procedure_code]
#                         rules_procedure_code = res["Procedure_Code"].values[0]
#                         rules_diff_in_days = res["Freq_Threshold_In_Days"].values[0]

#                         if rules_procedure_code == procedure_code and int(rules_diff_in_days) <= (diff_in_days):
#                             recommendation_procedure_list[procedure_code] = "1"
#                         else:
#                             recommendation_procedure_list[procedure_code] = "0"
#                     # if procedure_code == aic_procedure_code:
#                     #     if (rules_diff_in_days) <= (diff_in_days) and result_value < "7":
#                     #         recommendation_procedure_list[procedure_code] = "1"
#                     #     elif (rules_diff_in_days) <= (diff_in_days) and result_value < "8":
#                     #         recommendation_procedure_list[procedure_code] = "1"
#                     #     elif (rules_diff_in_days) <= (diff_in_days) and result_value < "9":
#                     #         recommendation_procedure_list[procedure_code] = "1"
#                     #     else:
#                     #         recommendation_procedure_list[procedure_code] = "0"
#                     else:
#                         for procedure in unique_procedure_code:
#                             if procedure not in list(recommendation_procedure_list.keys()):
#                                 recommendation_procedure_list[procedure] = "1"

#         except Exception as e:
#             self.logger.exception(str(e))

#         return recommendation_procedure_list

#     def allproceducecode(self, rules_data):

#         rec_result = dict()
#         Standard_procedure_List = rules_data["Procedure_Code"].value_counts().keys().tolist()
#         for item in Standard_procedure_List:
#             rec_result[item] = "1"
#         # return {str(Patient_ID): rec_result}
#         return rec_result
#         return actual_data
        


In [6]:
# Patient_data['Service_Date_From_trial']=pd.to_datetime(Patient_data['Service_Date_From'])
# date = Patient_data['Service_Date_From_trial']
# Patient_data['Min']=Patient_data['Service_Date_From_trial']
# remove = lambda x: Patient_data[x].duplicated(keep='last')  
# Patient_data.loc[remove('Service_Date_From_trial'), 'Service_Date_From_trial'] = np.nan
# p=Patient_data['Service_Date_From_trial'][:-1]
# p.index = p.index+1
# Patient_data['Max'] = pd.concat([pd.Series(['']),p]) 
# Patient_data['Min']=pd.to_datetime(Patient_data['Min'])
# Patient_data['Max']=pd.to_datetime(Patient_data['Max'])
# m = Patient_data['Max'].duplicated()
# Patient_data['Max'] = Patient_data['Max'].mask(m).ffill()
# Patient_data.drop('Service_Date_From_trial',axis=1,inplace=True)
# Patient_data

,Patient_ID,Service_Date_From,Procedure_Code,Primary_Diagnosis_Code,Min,Max
0,144681,2022-02-22,3008F,E11.65,2022-02-22,NaT
1,144681,2022-02-22,99214,E11.65,2022-02-22,NaT
2,144681,2022-02-22,83036,E11.65,2022-02-22,NaT
3,144681,2022-02-22,80053,E11.65,2022-02-22,NaT
4,144681,2022-02-22,36416,E11.65,2022-02-22,NaT
5,144681,2022-02-22,3079F,E11.65,2022-02-22,NaT
6,144681,2022-02-22,36415,E11.65,2022-02-22,NaT
7,144681,2022-02-22,3075F,E11.65,2022-02-22,NaT
8,144681,2022-02-22,3051F,E11.65,2022-02-22,NaT
9,144681,2021-08-23,36415,E11.65,2021-08-23,2022-02-22


In [7]:
# Patient_data["Service_Date_From"] = pd.to_datetime(Patient_data["Service_Date_From"])
# Patient_data["Service_Date_From"] =Patient_data["Service_Date_From"].astype(str)

In [8]:
# min_date,max_date=Patient_data["Service_Date_From"].min(),Patient_data["Service_Date_From"].max() 
# min_date,max_date

('2016-10-07', '2022-02-22')

In [9]:
# import datetime
# from dateutil import *
# formatter='%Y-%m-%d'

# lists_of_labcomp_data = Labresult_data["RESULT_COMPLETIONDATE"].unique()
# lists_of_labcomp_data= [pd.to_datetime(str(i)) for i in lists_of_labcomp_data]
# lists_of_labcomp_data.sort()
# date_range = pd.date_range(min_date,max_date)
# date_range = [pd.to_datetime(str(i)) for i in date_range] #parser.parse(str(i)).strftime('%Y-%m-%d')
# results = list(set(date_range).intersection(set(lists_of_labcomp_data)))
# results.sort(reverse=True)
# result_list = results

In [10]:
# resultant_df = pd.DataFrame()
# for i in range(len(result_list)):
#     res = Labresult_data.loc[Labresult_data["RESULT_COMPLETIONDATE"] == result_list[i]]
#     resultant_df = resultant_df.append(res)

In [11]:
# resultant_df

,IMREDEM_CODE,RESULT_ABNORMAL,Patient_Id,RESULT_NAME,RESULT_VALUE,RESULT_UNITS,RESULT_NORMAL_RANGE,RESULT_COMPLETIONDATE
1,87082,1,144681,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.6,%,3.5 - 6.0,2022-02-22
2,87082,1,144681,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.2,%,3.5 - 6.0,2021-08-23
3,87082,1,144681,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,8.1,%,3.5 - 6.0,2021-05-13
4,87082,1,144681,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.6,%,3.5 - 6.0,2021-02-11
5,87082,1,144681,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,10.4,%,3.5 - 6.0,2020-07-27
6,87082,1,144681,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.6,%,3.5 - 6.0,2020-04-23
7,87082,1,144681,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,6.8,%,3.5 - 6.0,2020-01-07
8,87082,1,144681,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,6.8,%,3.5 - 6.0,2019-10-08
9,87082,1,144681,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.2,%,3.5 - 6.0,2019-07-08
10,87082,1,144681,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,8.8,%,3.5 - 6.0,2019-04-08


In [12]:
# Patient_data['LabResult'] = None

In [13]:
# import datetime
# pd.to_datetime(1646092800000000000)
# lab_result_dates = list(set(resultant_df['RESULT_COMPLETIONDATE'].values.tolist()))
# lab_result_dates_new = [pd.to_datetime(l) for l in lab_result_dates]
# lab_result_dates_new.sort(reverse=True)
# Lab_result_list1 =lab_result_dates_new

In [14]:
# from datetime import date
# df_li = []
# today = date.today()
# str(today)
# for index,rows in Patient_data.iterrows():
#     status = 0
#     if rows['Procedure_Code'] == "83036":
#         min_date = rows['Min']
#         max_date = rows['Max']
#         if str(max_date) == "NaT":
#             max_date = min_date
#             now =  date.today()
#             rows['Max'] = now

#         for each_result_date in Lab_result_list1:
#             each_result_date = pd.to_datetime(each_result_date)
#             if each_result_date >= min_date and each_result_date <= max_date:
#                 rows['LabResult'] = each_result_date
#                 df_li.append(rows)
#                 df_li.append(rows.copy())
#                 status=1
#     if status==0:
#         df_li.append(rows)
# Patient_data_new = pd.DataFrame(df_li)
# Patient_data_new['RESULT_COMPLETIONDATE'] = Patient_data_new['LabResult'].apply(lambda x: x.strftime("%Y-%m-%d") if str(x)!= "NaT" else "NA")
# resultant_df['RESULT_COMPLETIONDATE'] = resultant_df['RESULT_COMPLETIONDATE'].apply(lambda x: x.strftime("%Y-%m-%d") if str(x)!= "NaT" else "NA")
# patient_df_lab_record_join = Patient_data_new.merge(resultant_df,on="RESULT_COMPLETIONDATE",how="left")
# #patient_df_lab_record_join=patient_df_lab_record_join.drop(['LabResult','IMREDEM_CODE','Patient_Id','RESULT_UNITS'],axis=1)
# #patient_df_lab_record_join.drop_duplicates(subset=['Patient_ID','Service_Date_From','Procedure_Code','Primary_Diagnosis_Code','Min','Max','RESULT_COMPLETIONDATE','RESULT_ABNORMAL','RESULT_NAME','RESULT_VALUE','RESULT_NORMAL_RANGE'],inplace=True)
        

In [16]:
# patient_df_lab_record_join 

,Patient_ID,Service_Date_From,Procedure_Code,Primary_Diagnosis_Code,Min,Max,LabResult,RESULT_COMPLETIONDATE,IMREDEM_CODE,RESULT_ABNORMAL,Patient_Id,RESULT_NAME,RESULT_VALUE,RESULT_UNITS,RESULT_NORMAL_RANGE
0,144681,2022-02-22,3008F,E11.65,2022-02-22,NaT,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,144681,2022-02-22,99214,E11.65,2022-02-22,NaT,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,144681,2022-02-22,83036,E11.65,2022-02-22,2022-07-13,2022-02-22,2022-02-22,87082.0,1.0,144681.0,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.6,%,3.5 - 6.0
3,144681,2022-02-22,83036,E11.65,2022-02-22,2022-07-13,2022-02-22,2022-02-22,87082.0,1.0,144681.0,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.6,%,3.5 - 6.0
4,144681,2022-02-22,80053,E11.65,2022-02-22,NaT,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,144681,2016-10-07,82044,E11.65,2016-10-07,2017-01-18,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,144681,2016-10-07,3046F,E11.65,2016-10-07,2017-01-18,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,144681,2016-10-07,99214,E11.65,2016-10-07,2017-01-18,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,144681,2016-10-07,3074F,E11.65,2016-10-07,2017-01-18,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# def getderiveddata(patient_df_lab_record_join,app_date):
#     import datetime
#     import datetime 
#     from datetime import datetime, time, date
#     #app_date="2022-07-15"
#     formatter = '%Y-%m-%d'
#     app_date = datetime.strptime(app_date, formatter)
#     patient_df_lab_record_join["Next_Appt_date"] = app_date
#     patient_df_lab_record_join["Service_Date_From"]=pd.to_datetime(patient_df_lab_record_join["Service_Date_From"])
#     #actual_data["Service_Date_To"]=pd.to_datetime(actual_data["Service_Date_To"])
    
#     patient_df_lab_record_join["Service_Date_max"] = patient_df_lab_record_join.groupby("Patient_ID")["Service_Date_From"].transform(max)
#     #patient_df_lab_record_join["new_date"]=pd.to_datetime(patient_df_lab_record_join["new_date"],errors='coerce')
#     #actual_data["last_visit_day"] = (actual_data["new_date"] - actual_data["Service_Date_max"])
#     #actual_data["last_visit_day"] = actual_data["last_visit_day"].apply(lambda x: int(str(x).split(" ")[0]))
#     #actual_data["Patient_Practice"] = actual_data.loc[:, "last_visit_day"].apply(lambda x: "Adhoc" if x >= 90 else "Regular")
#     patient_df_lab_record_join["Diff_in_days"] = (patient_df_lab_record_join["Next_Appt_date"] - patient_df_lab_record_join["Service_Date_max"])
#     patient_df_lab_record_join["Diff_in_days"] = patient_df_lab_record_join["Diff_in_days"].apply(lambda x: int(str(x).split(" ")[0]))
#     patient_df_lab_record_join=patient_df_lab_record_join[['Patient_ID','Service_Date_From','Procedure_Code','Primary_Diagnosis_Code','Next_Appt_date','Service_Date_max','Diff_in_days','RESULT_COMPLETIONDATE','RESULT_ABNORMAL','RESULT_NAME','RESULT_VALUE','RESULT_NORMAL_RANGE']]
#     return patient_df_lab_record_join

In [18]:
# patient_df_lab_record_join=getderiveddata(patient_df_lab_record_join,'2022-03-30') 
# patient_df_lab_record_join

,Patient_ID,Service_Date_From,Procedure_Code,Primary_Diagnosis_Code,Next_Appt_date,Service_Date_max,Diff_in_days,RESULT_COMPLETIONDATE,RESULT_ABNORMAL,RESULT_NAME,RESULT_VALUE,RESULT_NORMAL_RANGE
0,144681,2022-02-22,3008F,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
1,144681,2022-02-22,99214,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
2,144681,2022-02-22,83036,E11.65,2022-03-30,2022-02-22,36,2022-02-22,1.0,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.6,3.5 - 6.0
3,144681,2022-02-22,83036,E11.65,2022-03-30,2022-02-22,36,2022-02-22,1.0,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.6,3.5 - 6.0
4,144681,2022-02-22,80053,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
64,144681,2016-10-07,82044,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
65,144681,2016-10-07,3046F,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
66,144681,2016-10-07,99214,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
67,144681,2016-10-07,3074F,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN


In [66]:
# rules_data=pd.read_csv(r"E:/OneDrive - Quadratic Insights Pvt Ltd/Desktop/Epic_Rules.csv")
# rules_data

,Diabetic_Patient,Diagnosis_Code,Procedures_Description,Procedure_Code,Freq_Threshold_In_Days
0,Diabetic,E08-E13,HBA1C,83036,90
1,Diabetic,E08-E13,Urinalysis,81002,180
2,Diabetic,E08-E13,Microalbumin w/ Creatinine,82044,180
3,Diabetic,E08-E13,Microalbumin w/ Creatinine,82570,180
4,Diabetic,E08-E13,CMP (comprehensive metabolic panel),80053,90
5,Diabetic,E08-E13,Lipid,80061,120
6,Diabetic,E08-E13,TSH,84443,180
7,Diabetic,E08-E13,Foot exam,2028F,365
8,Diabetic,E08-E13,RETINAL EYE EXAM,92250,365
9,Diabetic,E08-E13,DILATED EYE EXAMINATION PERFORMED,2022F,365


In [67]:
# patient_df_lab_record_join

,Patient_ID,Service_Date_From,Procedure_Code,Primary_Diagnosis_Code,Next_Appt_date,Service_Date_max,Diff_in_days,RESULT_COMPLETIONDATE,RESULT_ABNORMAL,RESULT_NAME,RESULT_VALUE,RESULT_NORMAL_RANGE
0,144681,2022-02-22,3008F,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
1,144681,2022-02-22,99214,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
2,144681,2022-02-22,83036,E11.65,2022-03-30,2022-02-22,36,2022-02-22,1.0,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.6,3.5 - 6.0
4,144681,2022-02-22,80053,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
5,144681,2022-02-22,36416,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
6,144681,2022-02-22,3079F,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
7,144681,2022-02-22,36415,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
8,144681,2022-02-22,3075F,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
9,144681,2022-02-22,3051F,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN
10,144681,2021-08-23,36415,E11.65,2022-03-30,2022-02-22,36,NA,NaN,NaN,NaN,NaN


In [72]:
# for key, value in patient_df_lab_record_join.items():
#     print(key, value)

Patient_ID 0     144681
1     144681
2     144681
4     144681
5     144681
6     144681
7     144681
8     144681
9     144681
10    144681
11    144681
12    144681
13    144681
14    144681
15    144681
16    144681
17    144681
18    144681
19    144681
20    144681
21    144681
22    144681
23    144681
24    144681
25    144681
26    144681
28    144681
29    144681
30    144681
31    144681
32    144681
33    144681
34    144681
36    144681
38    144681
41    144681
42    144681
43    144681
44    144681
45    144681
46    144681
47    144681
48    144681
49    144681
50    144681
53    144681
54    144681
55    144681
56    144681
57    144681
58    144681
59    144681
60    144681
63    144681
64    144681
65    144681
66    144681
67    144681
68    144681
Name: Patient_ID, dtype: int64
Service_Date_From 0    2022-02-22
1    2022-02-22
2    2022-02-22
4    2022-02-22
5    2022-02-22
6    2022-02-22
7    2022-02-22
8    2022-02-22
9    2022-02-22
10   2021-08-23
11   2021-08-

In [73]:
print(len(patient_df_lab_record_join))
#patient_df_lab_record_join['Procedure_Code']=patient_df_lab_record_join['Procedure_Code'].astype(str)

59


In [71]:
#unique_procedure_code = rules_data["Procedure_Code"].value_counts().keys().tolist()
for row in range(len(patient_df_lab_record_join)):
    procedure_code_list = []
    diff_in_days_list = []
    procedure_code_list.append(patient_df_lab_record_join.loc[row, "Procedure_Code"])
    diff_in_days_list.append(patient_df_lab_record_join.loc[row, "Diff_in_days"])
    print(procedure_code_list)

['3008F']
['99214']
['83036']


KeyError: 3

In [20]:
def recommendation(one_patient_record, rules_data):
    recommendation_procedure_list = {}
            # aic_procedure_code = '3046f'
    unique_procedure_code = rules_data["Procedure_Code"].value_counts().keys().tolist()

    for row in range(len(one_patient_record)):
        procedure_code_list = []
        diff_in_days_list = []
                # result_value = []

        procedure_code_list.append(one_patient_record.loc[row, "Procedure_Code"])
        diff_in_days_list.append(one_patient_record.loc[row, "Diff_in_days"])
                #result_value.append(one_patient_record.loc[row, "RESULT_VALUE"])

        for record in range(len(procedure_code_list)):
            procedure_code = procedure_code_list[record]
            diff_in_days = diff_in_days_list[record]

            if procedure_code in unique_procedure_code:
                res = rules_data.loc[rules_data["Procedure_Code"] == procedure_code]
                rules_procedure_code = res["Procedure_Code"].values[0]
                rules_diff_in_days = res["Freq_Threshold_In_Days"].values[0]

                if rules_procedure_code == procedure_code and int(rules_diff_in_days) <= (diff_in_days):
                    recommendation_procedure_list[procedure_code] = "1"
                else:
                    recommendation_procedure_list[procedure_code] = "0"
                    
            else:
                for procedure in unique_procedure_code:
                    if procedure not in list(recommendation_procedure_list.keys()):
                        recommendation_procedure_list[procedure] = "1"


    return recommendation_procedure_list


In [21]:
recommendation(patient_df_lab_record_join, rules_data)

KeyError: 3

In [21]:
#Patient_data.groupby(by=['Service_Date_From','Patient_ID','Primary_Diagnosis_Code','Procedure_Code'], sort = False, as_index = False).nth([-1])

In [145]:
# a = (Patient_data.groupby(by=['Service_Date_From','Patient_ID','Primary_Diagnosis_Code','Procedure_Code']).last()).sort_values(by=['Service_Date_From'],ascending=False).reset_index()
# b=a.drop_duplicates(subset=['Service_Date_From'], keep='last')

In [149]:
b['Service_Date_From_trial']=b['Service_Date_From']

In [150]:
Patient_data.merge(b,on='Service_Date_From',how='left')

,Patient_ID_x,Service_Date_From,Primary_Diagnosis_Code_x,Procedure_Code_x,Patient_ID_y,Primary_Diagnosis_Code_y,Procedure_Code_y,Service_Date_From_trial
0,85622,2022-01-18,E11.65,36416,85622,E11.65,2022F,2022-01-18
1,85622,2022-01-18,E11.65,81002,85622,E11.65,2022F,2022-01-18
2,85622,2022-01-18,E11.65,82044,85622,E11.65,2022F,2022-01-18
3,85622,2022-01-18,E11.65,82570,85622,E11.65,2022F,2022-01-18
4,85622,2022-01-18,E11.65,82962,85622,E11.65,2022F,2022-01-18
5,85622,2022-01-18,E11.65,2022F,85622,E11.65,2022F,2022-01-18
6,85622,2022-01-18,E11.65,2026F,85622,E11.65,2022F,2022-01-18
7,85622,2022-01-18,E11.65,3046F,85622,E11.65,2022F,2022-01-18
8,85622,2022-01-18,E11.65,83036,85622,E11.65,2022F,2022-01-18
9,85622,2022-01-18,E11.65,92250,85622,E11.65,2022F,2022-01-18


In [118]:
service = Patient_data["Service_Date_From"]
service

0     2022-01-18
1     2022-01-18
2     2022-01-18
3     2022-01-18
4     2022-01-18
5     2022-01-18
6     2022-01-18
7     2022-01-18
8     2022-01-18
9     2022-01-18
10    2022-01-18
11    2021-12-27
12    2021-12-27
13    2021-12-27
14    2021-12-27
15    2021-12-27
16    2021-12-27
17    2021-06-07
18    2021-06-07
19    2021-06-07
20    2019-01-02
21    2019-01-02
22    2019-01-02
23    2019-01-02
24    2019-01-02
25    2019-01-02
26    2019-01-02
27    2019-01-02
Name: Service_Date_From, dtype: object

In [25]:
Patient_data["Service_Date_From"] = pd.to_datetime(Patient_data["Service_Date_From"])
Patient_data["Service_Date_From"] =Patient_data["Service_Date_From"].astype(str)

In [8]:
min_date,max_date=Patient_data["Service_Date_From"].min(),Patient_data["Service_Date_From"].max() 
min_date,max_date

('2019-01-02', '2022-01-18')

In [9]:
pd.date_range(min_date,max_date)

DatetimeIndex(['2019-01-02', '2019-01-03', '2019-01-04', '2019-01-05',
               '2019-01-06', '2019-01-07', '2019-01-08', '2019-01-09',
               '2019-01-10', '2019-01-11',
               ...
               '2022-01-09', '2022-01-10', '2022-01-11', '2022-01-12',
               '2022-01-13', '2022-01-14', '2022-01-15', '2022-01-16',
               '2022-01-17', '2022-01-18'],
              dtype='datetime64[ns]', length=1113, freq='D')

In [10]:
pip install parser

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement parser (from versions: none)
ERROR: No matching distribution found for parser
You should consider upgrading via the 'C:\Users\ssrampur\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [11]:
import parser
from dateutil import *
lists_of_labcomp_data = Labresult_data["RESULT_COMPLETIONDATE"].unique()
lists_of_labcomp_data= [parser.parse(str(i)).strftime('%Y-%m-%d') for i in lists_of_labcomp_data]
lists_of_labcomp_data.sort()
date_range = pd.date_range(min_date,max_date)
date_range = [parser.parse(str(i)).strftime('%Y-%m-%d') for i in date_range]
results = list(set(date_range).intersection(set(lists_of_labcomp_data)))

In [12]:
results.sort(reverse=True)
result_list = results
result_list

['2022-01-18',
 '2021-06-07',
 '2021-02-25',
 '2020-08-26',
 '2020-05-26',
 '2020-02-25',
 '2019-12-05',
 '2019-09-11',
 '2019-06-17',
 '2019-03-14',
 '2019-01-02']

In [13]:
resultant_df = pd.DataFrame()
for i in range(len(result_list)):
    res = Labresult_data.loc[Labresult_data["RESULT_COMPLETIONDATE"] == result_list[i]]
    resultant_df = resultant_df.append(res)
resultant_df

,IMREDEM_CODE,RESULT_ABNORMAL,Patient_Id,RESULT_NAME,RESULT_VALUE,RESULT_UNITS,RESULT_NORMAL_RANGE,RESULT_COMPLETIONDATE
2,2876,1,85622,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,9.5,%,3.5 - 6.0,2022-01-18
3,2876,1,85622,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,8.1,%,3.5 - 6.0,2021-06-07
4,2876,1,85622,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.7,%,3.5 - 6.0,2021-02-25
5,2876,1,85622,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,11.0,%,3.5 - 6.0,2020-08-26
6,2876,1,85622,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,12.8,%,3.5 - 6.0,2020-05-26
7,2876,1,85622,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,10.6,%,3.5 - 6.0,2020-02-25
8,2876,1,85622,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,12.1,%,3.5 - 6.0,2019-12-05
9,2876,1,85622,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,10.2,%,3.5 - 6.0,2019-09-11
10,2876,1,85622,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.0,%,3.5 - 6.0,2019-06-17
11,2876,1,85622,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,7.5,%,3.5 - 6.0,2019-03-14


In [14]:
import numpy as np 
Patient_data['Min']=Patient_data['Service_Date_From']
p=Patient_data['Min'][:-1]
p.index = p.index+1
Patient_data['Max'] = pd.concat([pd.Series(['']),p]) 
Patient_data['Min']=pd.to_datetime(Patient_data['Min'])
Patient_data['Max']=pd.to_datetime(Patient_data['Max'])

In [15]:
Patient_data.head(25)

,Patient_ID,Service_Date_From,Primary_Diagnosis_Code,Procedure_Code,Min,Max
0,85622,2022-01-18,E11.65,36416,2022-01-18,NaT
1,85622,2022-01-18,E11.65,81002,2022-01-18,2022-01-18
2,85622,2022-01-18,E11.65,82044,2022-01-18,2022-01-18
3,85622,2022-01-18,E11.65,82570,2022-01-18,2022-01-18
4,85622,2022-01-18,E11.65,82962,2022-01-18,2022-01-18
5,85622,2022-01-18,E11.65,2022F,2022-01-18,2022-01-18
6,85622,2022-01-18,E11.65,2026F,2022-01-18,2022-01-18
7,85622,2022-01-18,E11.65,3046F,2022-01-18,2022-01-18
8,85622,2022-01-18,E11.65,83036,2022-01-18,2022-01-18
9,85622,2022-01-18,E11.65,92250,2022-01-18,2022-01-18


In [16]:
Patient_data['LabResult'] = None

In [17]:
import datetime
pd.to_datetime(1646092800000000000)
lab_result_dates = list(set(resultant_df['RESULT_COMPLETIONDATE'].values.tolist()))
lab_result_dates_new = [pd.to_datetime(l) for l in lab_result_dates]


In [18]:

df_li = []
for index,rows in Patient_data.iterrows():
    status = 0
    min_date = rows['Min']
    max_date = rows['Max']
    if str(max_date) == "NaT":
        max_date = min_date
        rows['Max'] = max_date
        
    for each_result_date in lab_result_dates:
        each_result_date = pd.to_datetime(each_result_date)
        if each_result_date >= min_date and each_result_date < max_date:
            rows['LabResult'] = each_result_date
            df_li.append(rows)
            #df_li.append(rows.copy())
            status=1
    if status==0:
        df_li.append(rows)
Patient_data_new = pd.DataFrame(df_li)

Patient_data_new['RESULT_COMPLETIONDATE'] = Patient_data_new['LabResult'].apply(lambda x: x.strftime("%Y-%m-%d") if str(x)!= "NaT" else "NA")

resultant_df['RESULT_COMPLETIONDATE'] = resultant_df['RESULT_COMPLETIONDATE'].apply(lambda x: x.strftime("%Y-%m-%d") if str(x)!= "NaT" else "NA")
patient_df_lab_record_join = Patient_data_new.merge(resultant_df,on="RESULT_COMPLETIONDATE",how="left")


In [19]:
patient_df_lab_record_join=patient_df_lab_record_join.drop(['LabResult','Patient_Id','IMREDEM_CODE','RESULT_UNITS'],axis=1)
patient_df_lab_record_join

,Patient_ID,Service_Date_From,Primary_Diagnosis_Code,Procedure_Code,Min,Max,RESULT_COMPLETIONDATE,RESULT_ABNORMAL,RESULT_NAME,RESULT_VALUE,RESULT_NORMAL_RANGE
0,85622,2022-01-18,E11.65,36416,2022-01-18,2022-01-18,NA,NaN,NaN,NaN,NaN
1,85622,2022-01-18,E11.65,81002,2022-01-18,2022-01-18,NA,NaN,NaN,NaN,NaN
2,85622,2022-01-18,E11.65,82044,2022-01-18,2022-01-18,NA,NaN,NaN,NaN,NaN
3,85622,2022-01-18,E11.65,82570,2022-01-18,2022-01-18,NA,NaN,NaN,NaN,NaN
4,85622,2022-01-18,E11.65,82962,2022-01-18,2022-01-18,NA,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
31,85622,2019-01-02,E11.9,3075F,2019-01-02,2019-01-02,NA,NaN,NaN,NaN,NaN
32,85622,2019-01-02,E11.9,3078F,2019-01-02,2019-01-02,NA,NaN,NaN,NaN,NaN
33,85622,2019-01-02,E11.9,36416,2019-01-02,2019-01-02,NA,NaN,NaN,NaN,NaN
34,85622,2019-01-02,E11.9,81002,2019-01-02,2019-01-02,NA,NaN,NaN,NaN,NaN


In [ ]:
patient_df_lab_record_join.dropna()

In [95]:
import datetime 
from datetime import datetime, time, date
date_input ='2022-05-02'
format = '%Y-%m-%d'
datetime = datetime.strptime(date_input, format)
Patient_data["Next_Appointment_Date"] = datetime
Patient_data["Next_Appointment_Date"] = pd.to_datetime(Patient_data["Next_Appointment_Date"], format='%y%m%d')
Patient_data 

,Patient_ID,Service_Date_From,Primary_Diagnosis_Code,Procedure_Code,Min,Max,LabResult,Next_Appointment_Date
0,174588,2022-05-24,E11.65,99214,2022-05-24,NaT,None,2022-05-02
1,174588,2022-03-25,E11.42,99214,2022-03-25,2022-05-24,None,2022-05-02
2,174588,2022-03-25,E11.65,G0506,2022-03-25,2022-03-25,None,2022-05-02
3,174588,2022-03-25,E11.65,36416,2022-03-25,2022-03-25,None,2022-05-02
4,174588,2022-03-25,E11.65,85018,2022-03-25,2022-03-25,None,2022-05-02
5,174588,2022-03-11,E11.65,95250,2022-03-11,2022-03-25,None,2022-05-02
6,174588,2022-03-11,E11.65,95251,2022-03-11,2022-03-11,None,2022-05-02
7,174588,2022-03-01,E11.42,99214,2022-03-01,2022-03-11,None,2022-05-02
8,174588,2022-03-01,E11.65,G0506,2022-03-01,2022-03-01,None,2022-05-02
9,174588,2022-03-01,E11.65,36416,2022-03-01,2022-03-01,None,2022-05-02


In [106]:
#Patient_data.dtypes

Patient_ID                         int64
Service_Date_From                 object
Primary_Diagnosis_Code            object
Procedure_Code                    object
Min                       datetime64[ns]
Max                       datetime64[ns]
LabResult                         object
dtype: object

In [99]:
#Patient_data['LabResult'] = None
#Patient_data.head()

,Patient_ID,Service_Date_From,Primary_Diagnosis_Code,Procedure_Code,Min,Max,LabResult
0,174588,2022-05-24,E11.65,99214,2022-05-24,NaT,None
1,174588,2022-03-25,E11.42,99214,2022-03-25,2022-05-24,None
2,174588,2022-03-25,E11.65,G0506,2022-03-25,2022-03-25,None
3,174588,2022-03-25,E11.65,36416,2022-03-25,2022-03-25,None
4,174588,2022-03-25,E11.65,85018,2022-03-25,2022-03-25,None


In [186]:
lab_result_dates = list(set(resultant_df['RESULT_COMPLETIONDATE'].values.tolist()))
lab_result_dates

[1646092800000000000, 1624924800000000000, 1623974400000000000]

In [187]:
lab_result_dates_new = [pd.to_datetime(l) for l in lab_result_dates]
lab_result_dates_new

[Timestamp('2022-03-01 00:00:00'),
 Timestamp('2021-06-29 00:00:00'),
 Timestamp('2021-06-18 00:00:00')]

In [188]:
import datetime

In [189]:
pd.to_datetime(1646092800000000000)

Timestamp('2022-03-01 00:00:00')

In [190]:
df_li = []
for index,rows in Patient_data.iterrows():
    status = 0
    min_date = rows['Min']
    max_date = rows['Max']
    if str(max_date) == "NaT":
        max_date = min_date
        rows['Max'] = max_date
        
    for each_result_date in lab_result_dates:
        each_result_date = pd.to_datetime(each_result_date)
        if each_result_date >= min_date and each_result_date < max_date:
            rows['LabResult'] = each_result_date
            df_li.append(rows.copy())
            status=1
    if status==0:
        df_li.append(rows)

Patient_data_new['RESULT_COMPLETIONDATE'] = Patient_data_new['LabResult'].apply(lambda x: x.strftime("%Y-%m-%d") if str(x)!= "NaT" else "NA")

resultant_df['RESULT_COMPLETIONDATE'] = resultant_df['RESULT_COMPLETIONDATE'].apply(lambda x: x.strftime("%Y-%m-%d") if str(x)!= "NaT" else "NA")
patient_df_lab_record_join = Patient_data_new.merge(resultant_df,on="RESULT_COMPLETIONDATE",how="left")

In [ ]:
patient_df_lab_record_join

In [191]:
Patient_data_new = pd.DataFrame(df_li)
Patient_data_new

,Patient_ID,Service_Date_From,Primary_Diagnosis_Code,Procedure_Code,Min,Max,LabResult
0,174588,2022-05-24,E11.65,99214,2022-05-24,2022-05-24,NaT
1,174588,2022-03-25,E11.42,99214,2022-03-25,2022-05-24,NaT
2,174588,2022-03-25,E11.65,G0506,2022-03-25,2022-03-25,NaT
3,174588,2022-03-25,E11.65,36416,2022-03-25,2022-03-25,NaT
4,174588,2022-03-25,E11.65,85018,2022-03-25,2022-03-25,NaT
5,174588,2022-03-11,E11.65,95250,2022-03-11,2022-03-25,NaT
6,174588,2022-03-11,E11.65,95251,2022-03-11,2022-03-11,NaT
7,174588,2022-03-01,E11.42,99214,2022-03-01,2022-03-11,2022-03-01
8,174588,2022-03-01,E11.65,G0506,2022-03-01,2022-03-01,NaT
9,174588,2022-03-01,E11.65,36416,2022-03-01,2022-03-01,NaT


In [161]:
resultant_df.head()

,IMREDEM_CODE,RESULT_ABNORMAL,Patient_Id,RESULT_NAME,RESULT_VALUE,RESULT_UNITS,RESULT_NORMAL_RANGE,RESULT_COMPLETIONDATE
0,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
1,110634,1,174588,HbA1c,9.1,#/HPF,3.5 - 6.0,2021-06-29
2,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,9.1,%,3.5 - 6.0,2021-06-18


In [192]:
Patient_data_new['RESULT_COMPLETIONDATE'] = Patient_data_new['LabResult'].apply(lambda x: x.strftime("%Y-%m-%d") if str(x)!= "NaT" else "NA")

In [193]:
resultant_df['RESULT_COMPLETIONDATE'] = resultant_df['RESULT_COMPLETIONDATE'].apply(lambda x: x.strftime("%Y-%m-%d") if str(x)!= "NaT" else "NA")

In [194]:
resultant_df.head()

,IMREDEM_CODE,RESULT_ABNORMAL,Patient_Id,RESULT_NAME,RESULT_VALUE,RESULT_UNITS,RESULT_NORMAL_RANGE,RESULT_COMPLETIONDATE
0,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
1,110634,1,174588,HbA1c,9.1,#/HPF,3.5 - 6.0,2021-06-29
2,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,9.1,%,3.5 - 6.0,2021-06-18


In [195]:
Patient_data_new

,Patient_ID,Service_Date_From,Primary_Diagnosis_Code,Procedure_Code,Min,Max,LabResult,RESULT_COMPLETIONDATE
0,174588,2022-05-24,E11.65,99214,2022-05-24,2022-05-24,NaT,NA
1,174588,2022-03-25,E11.42,99214,2022-03-25,2022-05-24,NaT,NA
2,174588,2022-03-25,E11.65,G0506,2022-03-25,2022-03-25,NaT,NA
3,174588,2022-03-25,E11.65,36416,2022-03-25,2022-03-25,NaT,NA
4,174588,2022-03-25,E11.65,85018,2022-03-25,2022-03-25,NaT,NA
5,174588,2022-03-11,E11.65,95250,2022-03-11,2022-03-25,NaT,NA
6,174588,2022-03-11,E11.65,95251,2022-03-11,2022-03-11,NaT,NA
7,174588,2022-03-01,E11.42,99214,2022-03-01,2022-03-11,2022-03-01,2022-03-01
8,174588,2022-03-01,E11.65,G0506,2022-03-01,2022-03-01,NaT,NA
9,174588,2022-03-01,E11.65,36416,2022-03-01,2022-03-01,NaT,NA


In [163]:
resultant_df.dtypes

IMREDEM_CODE              int64
RESULT_ABNORMAL           int64
Patient_Id                int64
RESULT_NAME              object
RESULT_VALUE             object
RESULT_UNITS             object
RESULT_NORMAL_RANGE      object
RESULT_COMPLETIONDATE    object
dtype: object

In [164]:
Patient_data_new.dtypes

Patient_ID                         int64
Service_Date_From                 object
Primary_Diagnosis_Code            object
Procedure_Code                    object
Min                       datetime64[ns]
Max                       datetime64[ns]
LabResult                 datetime64[ns]
RESULT_COMPLETIONDATE             object
dtype: object

In [196]:
patient_df_lab_record_join = Patient_data_new.merge(resultant_df,on="RESULT_COMPLETIONDATE",how="left")

In [197]:
patient_df_lab_record_join

,Patient_ID,Service_Date_From,Primary_Diagnosis_Code,Procedure_Code,Min,Max,LabResult,RESULT_COMPLETIONDATE,IMREDEM_CODE,RESULT_ABNORMAL,Patient_Id,RESULT_NAME,RESULT_VALUE,RESULT_UNITS,RESULT_NORMAL_RANGE
0,174588,2022-05-24,E11.65,99214,2022-05-24,2022-05-24,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,174588,2022-03-25,E11.42,99214,2022-03-25,2022-05-24,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,174588,2022-03-25,E11.65,G0506,2022-03-25,2022-03-25,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,174588,2022-03-25,E11.65,36416,2022-03-25,2022-03-25,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,174588,2022-03-25,E11.65,85018,2022-03-25,2022-03-25,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,174588,2022-03-11,E11.65,95250,2022-03-11,2022-03-25,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,174588,2022-03-11,E11.65,95251,2022-03-11,2022-03-11,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,174588,2022-03-01,E11.42,99214,2022-03-01,2022-03-11,2022-03-01,2022-03-01,110634.0,1.0,174588.0,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0
8,174588,2022-03-01,E11.65,G0506,2022-03-01,2022-03-01,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,174588,2022-03-01,E11.65,36416,2022-03-01,2022-03-01,NaT,NA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#x=Patient_data.iloc[2:,:]
#x=x.reset_index(drop=True)
#x 

In [89]:
df1 = pd.DataFrame()
df2 = pd.DataFrame()
starting_index = []
resultant_df['RESULT_COMPLETIONDATE']=pd.to_datetime(resultant_df['RESULT_COMPLETIONDATE'])
for index,rows in Patient_data.iterrows():
    min_date, min_index = (Patient_data.iloc[index,4]), index
    max_date, max_index = (Patient_data.iloc[index,5]), index
    result = pd.date_range(min_date, max_date)
    starting_index.append(min_index)
    
    result = [parser.parse(str(i)).strftime('%Y-%m-%d') for i in result]
    result_completion_date_df = resultant_df["RESULT_COMPLETIONDATE"].values
    result_completion_date_df = [parser.parse(str(i)).strftime('%Y-%m-%d') for i in result_completion_date_df]
#     result_completion_date_df
#     print(result)
    for i in range(len(result_completion_date_df)):
        if result_completion_date_df[i] in (result):
            new_df = resultant_df.loc[resultant_df["RESULT_COMPLETIONDATE"] == result_completion_date_df[i]]
#             Patient_data = pd.concat([Patient_data, new_df], axis = 0)
           
            
        
    #date_list=result.to_list(resultant_df['RESULT_COMPLETIONDATE'])
    #resultant_df.loc[]
    #r=resultant_df.loc[resultant_df['RESULT_COMPLETIONDATE'].isin(result)]
    #df1=df1.append(r) 
# Patient_data
starting_index

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2]

In [60]:
date_range = pd.date_range(min_date,max_date)

In [65]:
date_range = [parser.parse(str(i)).strftime('%Y-%m-%d') for i in date_range]

In [67]:
type(date_range)

list

In [75]:
result_completion_date_df = resultant_df["RESULT_COMPLETIONDATE"].values
result_completion_date_df = [parser.parse(str(i)).strftime('%Y-%m-%d') for i in result_completion_date_df]
result_completion_date_df

['2022-03-01', '2021-06-29', '2021-06-18']

In [12]:
final_res = pd.merge(Patient_data,resultant_df,left_on=['Patient_ID','Service_Date_From'],right_on=['Patient_Id','RESULT_COMPLETIONDATE'])
final_res

,Patient_ID,Service_Date_From,Primary_Diagnosis_Code,Procedure_Code,Appointment_DateTime,IMREDEM_CODE,RESULT_ABNORMAL,Patient_Id,RESULT_NAME,RESULT_VALUE,RESULT_UNITS,RESULT_NORMAL_RANGE,RESULT_COMPLETIONDATE
0,174588,2022-03-01,E11.42,99214,2022-03-01 08:40:00,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
1,174588,2022-03-01,E11.65,G0506,2022-03-01 08:40:00,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
2,174588,2022-03-01,E11.65,36416,2022-03-01 08:40:00,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
3,174588,2022-03-01,E11.65,81002,2022-03-01 08:40:00,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
4,174588,2022-03-01,E11.65,82044,2022-03-01 08:40:00,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
5,174588,2022-03-01,E11.65,82570,2022-03-01 08:40:00,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
6,174588,2022-03-01,E11.65,82962,2022-03-01 08:40:00,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
7,174588,2022-03-01,E11.65,83036,2022-03-01 08:40:00,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
8,174588,2022-03-01,E11.65,3046F,2022-03-01 08:40:00,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
9,174588,2021-06-18,E11.65,G9007,2021-06-18 15:40:00,110634,1,174588,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,9.1,%,3.5 - 6.0,2021-06-18


In [5]:
import datetime 
from datetime import datetime, time, date
date_input ='2022-05-02'
format = '%Y-%m-%d'
datetime = datetime.strptime(date_input, format)
Patient_data["Next_Appointment_Date"] = datetime
Patient_data["Next_Appointment_Date"] = pd.to_datetime(Patient_data["Next_Appointment_Date"], format='%y%m%d')
Patient_data

,Patient_ID,Service_Date_From,Primary_Diagnosis_Code,Procedure_Code,Appointment_DateTime,Next_Appointment_Date
0,174588,2021-05-21,E11.65,99214,2021-05-21 14:20:00,2022-05-02
1,174588,2021-06-18,E11.65,G9007,2021-06-18 15:40:00,2022-05-02
2,174588,2022-03-01,E11.42,99214,2022-03-01 08:40:00,2022-05-02
3,174588,2022-03-01,E11.65,G0506,2022-03-01 08:40:00,2022-05-02
4,174588,2022-03-01,E11.65,36416,2022-03-01 08:40:00,2022-05-02
5,174588,2022-03-01,E11.65,81002,2022-03-01 08:40:00,2022-05-02
6,174588,2022-03-01,E11.65,82044,2022-03-01 08:40:00,2022-05-02
7,174588,2022-03-01,E11.65,82570,2022-03-01 08:40:00,2022-05-02
8,174588,2022-03-01,E11.65,82962,2022-03-01 08:40:00,2022-05-02
9,174588,2022-03-01,E11.65,83036,2022-03-01 08:40:00,2022-05-02


In [6]:
Patient_data["Diff_In_days"]=(Patient_data["Next_Appointment_Date"]-Patient_data["Service_Date_From"])
Patient_data["Diff_In_days"] = Patient_data["Diff_In_days"].apply(lambda x : int(str(x).split(" ")[0]))
Patient_data["Service_Date_From"] = pd.to_datetime(Patient_data["Service_Date_From"])
Patient_data["Service_Date_From"] =Patient_data["Service_Date_From"].astype(str)
#3046F

In [7]:
df=pd.merge(Patient_data,Labresult_data,left_on=['Patient_ID','Service_Date_From'],right_on=['Patient_Id','RESULT_COMPLETIONDATE'])
df

,Patient_ID,Service_Date_From,Primary_Diagnosis_Code,Procedure_Code,Appointment_DateTime,Next_Appointment_Date,Diff_In_days,IMREDEM_CODE,Patient_Id,RESULT_ABNORMAL,RESULT_NAME,RESULT_VALUE,RESULT_UNITS,RESULT_NORMAL_RANGE,RESULT_COMPLETIONDATE
0,174588,2021-06-18,E11.65,G9007,2021-06-18 15:40:00,2022-05-02,318,110634,174588,1,HEMOGLOBIN GLYCLATED (HGB A1C)-IO,9.1,%,3.5 - 6.0,2021-06-18
1,174588,2022-03-01,E11.42,99214,2022-03-01 08:40:00,2022-05-02,62,110634,174588,1,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
2,174588,2022-03-01,E11.65,G0506,2022-03-01 08:40:00,2022-05-02,62,110634,174588,1,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
3,174588,2022-03-01,E11.65,36416,2022-03-01 08:40:00,2022-05-02,62,110634,174588,1,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
4,174588,2022-03-01,E11.65,81002,2022-03-01 08:40:00,2022-05-02,62,110634,174588,1,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
5,174588,2022-03-01,E11.65,82044,2022-03-01 08:40:00,2022-05-02,62,110634,174588,1,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
6,174588,2022-03-01,E11.65,82570,2022-03-01 08:40:00,2022-05-02,62,110634,174588,1,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
7,174588,2022-03-01,E11.65,82962,2022-03-01 08:40:00,2022-05-02,62,110634,174588,1,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
8,174588,2022-03-01,E11.65,83036,2022-03-01 08:40:00,2022-05-02,62,110634,174588,1,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01
9,174588,2022-03-01,E11.65,3046F,2022-03-01 08:40:00,2022-05-02,62,110634,174588,1,HEMOGLOBIN GLYCLATED (HGB A1C),9.5,%,3.5 - 6.0,2022-03-01


In [8]:
df1=df.loc[(df['Procedure_Code'] == '3046F') & (df['RESULT_ABNORMAL']==1)]

In [182]:
# Step 1: new_df[Procedure_Code]=3046F   --- No records Found
# Step 2: new_df[RESULT_ABNORMAL]==1
         # Recommend_procedure
# Step 3:IF FRE_OF_DAYS<30 and Result_value<9:
             #No need to recommed
        #ElSEIF FRE_OF_DAYS<60 and Result_value<8:
             #No need to recommed 
        #ELSEIF FRE_OF_DAYS<90 and Result_value<9:   
             #NO NEED TO RECOMMED
       # ELSE:
        #RECOMMEND 
    

In [135]:
def patient_details(Patient_ID,new_df):
    procedure_code = []
    Diff_in_days = []
    one_patient_data = new_df.loc[new_df["Patient_ID"] == Patient_ID]
    one_patient_data = one_patient_data.reset_index(drop=True)
    
    flag=True
    for index,rows in one_patient_data.iterrows():
        for row in range(len(rows)):
            if one_patient_data.iloc[index,9] != "0": 
                    flag=False
                    break             

    if flag:
       
        for index, rows in one_patient_data.iterrows():
            for key in rows.to_dict(): 
                if (key == "Procedure_Code"): 
                    procedure_code.append((rows[key]))

                if (key == "Diff_in_days"): 
                    Diff_in_days.append((rows[key]))
    return procedure_code, Diff_in_days

In [136]:
p=patient_details(174588,new_df)
p 

([], [])

In [69]:
Recommened_procedure=[]
procedure_code=new_df["Procedure_Code"].value_counts().keys().tolist()
if new_df[new_df['RESULT_ABNORMAL']==1]:
    Recommened_procedure.append[procedure_code] = "1"
else:
    pass

res = rules_data.loc[rules_data["Procedure_Code"] == procedure_code]

['3046F']

In [50]:
Diff_In_days= new_df["Diff_In_days"].value_counts().keys().tolist()


In [ ]:
If the patient is anbnormal then he/she needs to follow procedure
otherwise check condition with diff_In_days.

In [ ]:
IF FRE_OF_DAYS<30 and Result_value<9:
             No need to recommed
        ElSEIF FRE_OF_DAYS<60 and Result_value<8:
             No need to recommed 
        ELSEIF FRE_OF_DAYS<90 and Result_value<9:   
             NO NEED TO RECOMMED
        ELSE:
        RECOMMEND 

## PAtient_data

In [ ]:
import datetime
import logging
import sys
from datetime import datetime, time, date
import warnings
import pandas as pd
import numpy as np
from django.db import connection

warnings.filterwarnings("ignore")
formatter = '%Y-%m-%d'
logger = logging.getLogger(__name__)


class PatientData:
    def __init__(self):
        self.logger = logging.getLogger(__name__)
        self.sName = "Recommend Procedure Service"
        self.status = 0
        extra = {
            "cls_name": self.__class__.__name__,
        }
        self.logger = logging.LoggerAdapter(self.logger, extra)

    def get_patient_service_history(self, patient_id):
        try:

            cursor1 = connection.cursor()
            cursor1.execute("""Select [Patient_ID],[Service_Date_From],[Procedure_Code],
                            [Primary_Diagnosis_Code]
							from Ntier_EPIC.dbo.[vwGenSvcInfo] where Primary_Diagnosis_Code
                            between 'E08' and 'E13' and Patient_ID = %s """, (patient_id,))

            result1 = pd.DataFrame([list(elem) for elem in cursor1.fetchall()])

            if result1.empty:
                result_dict1 = {"response_key": "-1"}
            else:
                self.logger.info("result1 == " + str(result1))
                result1.columns = ['Patient_ID', 'Service_Date_From', 'Procedure_Code', 'Primary_Diagnosis_Code']
                result1.sort_values(by='Service_Date_From', inplace=True, ascending=False)
                result_dict1 = {"response_key": result1}

        except Exception as e:
            self.logger.exception(str(e))
            print("Exception  ", str(e))
            result_dict1 = {"response_key": "-1"}

        return result_dict1

    def get_patient_lab_result(self, patient_id):
        try:

            cursor2 = connection.cursor()

            cursor2.execute("""Select T3.Patient_ID,T1.IMREDEM_CODE,T1.RESULT_ABNORMAL, T1.RESULT_NAME, 
                                    T1.RESULT_VALUE, T1.RESULT_UNITS,T1.RESULT_NORMAL_RANGE,CONVERT(VARCHAR(10),T1.RESULT_COMPLETIONDATE, 111) AS RESULT_COMPLETIONDATE
									--CAST(T1.RESULT_COMPLETIONDATE as nvarchar(100)) RESULT_COMPLETIONDATE 
                                    FROM EMR.[HPSITE].LAB_RESULT AS T1 
                                    LEFT JOIN EMR.[HPSITE].[DEMOGRAPHICS] AS T2 ON T1.IMREDEM_CODE=T2.IMREDEM_CODE
                                    LEFT JOIN Ntier_EPIC.dbo.vwGenPatInfo T3 ON T2.DEM_EXTERNALID=T3.Patient_Number
                                    where (T1.RESULT_VALUE IS NOT NULL) and (T1.RESULT_NORMAL_RANGE IS NOT NULL) and T3.Patient_ID = %s """,
                            (patient_id,))

            result2 = pd.DataFrame([list(elem) for elem in cursor2.fetchall()])

            if result2.empty:
                result_dict2 = {"response_key": "-1"}
            else:
                result2.columns = ['Patient_ID', 'IMREDEM_CODE', 'RESULT_ABNORMAL', 'RESULT_NAME', 'RESULT_VALUE',
                                   'RESULT_UNITS', 'RESULT_NORMAL_RANGE', 'RESULT_COMPLETIONDATE']

                result2.sort_values(by='RESULT_COMPLETIONDATE', inplace=True, ascending=False)
                self.logger.info("result2 == " + str(result2))

                result_dict2 = {"response_key": result2}

        except Exception as e:
            self.logger.exception(str(e))
            result_dict2 = {"response_key": "-1"}

        return result_dict2

    def get_min_max_daterange(self, Patient_data, Labresult_data):
        try:
            Labresult_data["RESULT_COMPLETIONDATE"] = pd.to_datetime(Labresult_data["RESULT_COMPLETIONDATE"])
            Patient_data['Service_Date_From_trial'] = pd.to_datetime(Patient_data['Service_Date_From'])
            # date= Patient_data['Service_Date_From_trial']
            Patient_data['Min'] = Patient_data['Service_Date_From_trial']
            remove = lambda x: Patient_data[x].duplicated(keep='last')
            Patient_data.loc[remove('Service_Date_From_trial'), 'Service_Date_From_trial'] = np.nan
            p = Patient_data['Service_Date_From_trial'][:-1]
            p.index = p.index + 1
            Patient_data['Max'] = pd.concat([pd.Series(['']), p])
            Patient_data['Min'] = pd.to_datetime(Patient_data['Min'])
            Patient_data['Max'] = pd.to_datetime(Patient_data['Max'])
            m = Patient_data['Max'].duplicated()
            Patient_data['Max'] = Patient_data['Max'].mask(m).ffill()
            Patient_data.drop('Service_Date_From_trial', axis=1, inplace=True)
            Patient_data["Service_Date_From"] = pd.to_datetime(Patient_data["Service_Date_From"])
            Patient_data["Service_Date_From"] = Patient_data["Service_Date_From"].astype(str)
            min_date, max_date = Patient_data["Service_Date_From"].min(), Patient_data["Service_Date_From"].max()
            lists_of_labcomp_data = Labresult_data["RESULT_COMPLETIONDATE"].unique()
            lists_of_labcomp_data = [pd.to_datetime(str(i)) for i in lists_of_labcomp_data]
            lists_of_labcomp_data.sort()
            date_range = pd.date_range(min_date, max_date)
            date_range = [pd.to_datetime(str(i)) for i in date_range]  # parser.parse(str(i)).strftime('%Y-%m-%d')
            results = list(set(date_range).intersection(set(lists_of_labcomp_data)))
            results.sort(reverse=True)
            result_list = results
            resultant_df = pd.DataFrame()
            for i in range(len(result_list)):
                res = Labresult_data.loc[Labresult_data["RESULT_COMPLETIONDATE"] == result_list[i]]
                resultant_df = resultant_df.append(res)
            return Patient_data, Labresult_data, resultant_df

        except Exception as e:
            self.logger.exception(str(e))

    def get_final_df_with_lab_results(self, Patient_data, resultant_df):
        try:
            df_li = []
            today = date.today()
            str(today)
            Patient_data['LabResult'] = None
            lab_result_dates = list(set(resultant_df['RESULT_COMPLETIONDATE'].values.tolist()))
            lab_result_dates_new = [pd.to_datetime(l) for l in lab_result_dates]
            lab_result_dates_new.sort(reverse=True)
            Lab_result_list1 = lab_result_dates_new

            for index, rows in Patient_data.iterrows():
                status = 0
                if rows['Procedure_Code'] == "83036":
                    min_date = rows['Min']
                    max_date = rows['Max']
                    if str(max_date) == "NaT":
                        max_date = min_date
                        now = date.today()
                        rows['Max'] = now

                    for each_result_date in Lab_result_list1:
                        each_result_date = pd.to_datetime(each_result_date)
                        if each_result_date >= min_date and each_result_date <= max_date:
                            rows['LabResult'] = each_result_date
                            df_li.append(rows)
                            df_li.append(rows.copy())
                            status = 1
                if status == 0:
                    df_li.append(rows)

            Patient_data_new = pd.DataFrame(df_li)
            Patient_data_new['RESULT_COMPLETIONDATE'] = Patient_data_new['LabResult'].apply(
                lambda x: x.strftime("%Y-%m-%d") if str(x) != "NaT" else "NA")
            resultant_df['RESULT_COMPLETIONDATE'] = resultant_df['RESULT_COMPLETIONDATE'].apply(
                lambda x: x.strftime("%Y-%m-%d") if str(x) != "NaT" else "NA")
            patient_df_lab_record_join = Patient_data_new.merge(resultant_df, on="RESULT_COMPLETIONDATE", how="left")
            return patient_df_lab_record_join, resultant_df

        except Exception as e:
            self.logger.exception(str(e))

    def get_patient_derived_data(self, patient_df_lab_record_join, app_date):
        try:
            app_date = datetime.strptime(app_date, formatter)
            patient_df_lab_record_join.reset_index(drop=True)
            patient_df_lab_record_join["Next_Appt_date"] = app_date
            patient_df_lab_record_join["Service_Date_From"] = pd.to_datetime(
                patient_df_lab_record_join["Service_Date_From"])
            patient_df_lab_record_join["Service_Date_max"] = patient_df_lab_record_join.groupby("Patient_ID_x")[
                "Service_Date_From"].transform(max)
            patient_df_lab_record_join["Diff_in_days"] = (
                    patient_df_lab_record_join["Next_Appt_date"] - patient_df_lab_record_join["Service_Date_max"])
            patient_df_lab_record_join["Diff_in_days"] = patient_df_lab_record_join["Diff_in_days"].apply(
                lambda x: int(str(x).split(" ")[0]))
            patient_df_lab_record_join["Patient_Practice"] = patient_df_lab_record_join.loc[:, "Diff_in_days"].apply(
                lambda x: "Adhoc" if x >= 90 else "Regular")
            patient_df_lab_record_join = patient_df_lab_record_join[
                ['Patient_ID_x', 'Service_Date_From', 'Procedure_Code', 'Primary_Diagnosis_Code', 'Next_Appt_date',
                 'Service_Date_max', 'Diff_in_days', 'Patient_Practice', 'RESULT_COMPLETIONDATE', 'RESULT_ABNORMAL',
                 'RESULT_NAME', 'RESULT_VALUE', 'RESULT_NORMAL_RANGE']]
            return patient_df_lab_record_join

        except Exception as e:
            self.logger.exception(str(e))

    def recommendation(self, one_patient_record, rules_data):
        try:
            recommendation_procedure_list = {}
            aic_procedure_code = '83036'
            aic_procedure_code_value = list(one_patient_record[one_patient_record['Procedure_Code'] == '83036']['RESULT_VALUE'])[0]
            unique_procedure_code = rules_data["Procedure_Code"].value_counts().keys().tolist()

            for row in range(len(one_patient_record)):
                procedure_code_list = []
                diff_in_days_list = []
                result_value = []

                procedure_code_list.append(one_patient_record.loc[row, "Procedure_Code"])
                diff_in_days_list.append(one_patient_record.loc[row, "Diff_in_days"])
                result_value.append(one_patient_record.loc[row, "RESULT_VALUE"])

                for record in range(len(procedure_code_list)):
                    procedure_code = procedure_code_list[record]
                    diff_in_days = diff_in_days_list[record]

                    if procedure_code in unique_procedure_code:
                        res = rules_data.loc[rules_data["Procedure_Code"] == procedure_code]
                        rules_procedure_code = res["Procedure_Code"].values[0]
                        rules_diff_in_days = res["Freq_Threshold_In_Days"].values[0]

                        if rules_procedure_code == procedure_code and int(rules_diff_in_days) <= (diff_in_days):
                            recommendation_procedure_list[procedure_code] = "1"
                        else:
                            recommendation_procedure_list[procedure_code] = "0"

                        if procedure_code == aic_procedure_code:
                            if rules_diff_in_days >= diff_in_days and aic_procedure_code_value > '7.0':
                                recommendation_procedure_list[procedure_code] = "1"
                            elif rules_diff_in_days >= diff_in_days and aic_procedure_code_value > '8.0':
                                recommendation_procedure_list[procedure_code] = "1"
                            elif rules_diff_in_days >= diff_in_days and aic_procedure_code_value > '9.0':
                                recommendation_procedure_list[procedure_code] = "1"
                            else:
                                recommendation_procedure_list[procedure_code] = "0"

                    else:
                        for procedure in unique_procedure_code:
                            if procedure not in list(recommendation_procedure_list.keys()):
                                recommendation_procedure_list[procedure] = "1"

            return recommendation_procedure_list
        except Exception as e:
            self.logger.exception(str(e))

    def allproceducecode(self, rules_data):
        try:
            rec_result = dict()
            Standard_procedure_List = rules_data["Procedure_Code"].value_counts().keys().tolist()
            for item in Standard_procedure_List:
                rec_result[item] = "1"
            return rec_result

        except Exception as e:
            self.logger.exception(str(e))

            # exception_type, exception_object, exception_traceback = sys.exc_info()
            # filename = exception_traceback.tb_frame.f_code.co_filename
            # line_number = exception_traceback.tb_lineno

            # print("Exception type: ", exception_type)
            # print("File name: ", filename)
            # print("Line number: ", line_number)

    #         if Patient_data.loc[0]["RESULT_ABNORMAL"] == 1:
    #             Patient_data["Reccomend_Code"] = Patient_data["Procedure_Code"]
    #
    #         elif (Patient_data.loc[0]["Diff_In_days"] < 90) and (Patient_data.loc[0]["RESULT_VALUE"] < 7):
    #             Patient_data["Reccomend_Code"] = Patient_data["Procedure_Code"]
    #
    #         elif (Patient_data.loc[0]["Diff_In_days"] < 60) and (Patient_data.loc[0]["RESULT_VALUE"] < 8):
    #             Patient_data["Reccomend_Code"] = Patient_data["Procedure_Code"]
    #
    #
    #         elif (Patient_data.loc[0]["Diff_In_days"] < 30) and (Patient_data.loc[0]["RESULT_VALUE"] < 9):
    #             Patient_data["Reccomend_Code"] = Patient_data["Procedure_Code"]
    #
    #         else:
    #             Patient_data["Reccomend_Code"] = 0
    #
    #         return Patient_data


## Views

In [ ]:
import logging
import os
import time
from configparser import ConfigParser

import pandas as pd
from django.conf import settings
from django.http import HttpResponse
from rest_framework.response import Response
from rest_framework.views import APIView
from .InputRequestValidation import InputRequestValidation
from .PatientData import PatientData
from .ResponseJson import ResponseJson
from .errorcode import ErrorCodes

config = ConfigParser()
config.read("config/config.ini")

log_dir = settings.LOG_DIR
rules_data = pd.read_csv(config["path"]["rules_data_path"])


def log_file(self):
    try:
        file_name = "RuleEngine" + ".log"
        f = open(os.path.join(log_dir, file_name), "r")
        file_contents = f.read()
        f.close()
        return HttpResponse(file_contents, content_type="text/plain")
    except Exception as e:
        return HttpResponse(str(e), content_type="text/plain")


class RecommendProcedure(APIView):
    """
        This View takes two numbers as input and does addition of two numbers
        and provides the sum as output
    """

    def __init__(self):
        self.logger = logging.getLogger(__name__)
        self.errorObj = ErrorCodes()
        self.responseObj = ResponseJson()
        self.s_Name = "Recommend Procedure Service"
        self.status = 0
        self.patient_practice = "Adhoc"
        self.patient_id = "-1"
        self.recommended_code = {}
        extra = {
            "cls_name": self.__class__.__name__,
        }
        self.logger = logging.LoggerAdapter(self.logger, extra)

    def post(self, request):
        start_time = time.time()
        request_data = request.data
        input_request_validation_obj = InputRequestValidation()
        try:
            # req_val_res = input_request_validation_obj.request_validation(request_data)
            self.logger.info("recommendation Started")

            patient_id = request_data["Patient_ID"]
            latest_app_date = request_data["Latest_Appointment_Date"]

            patient_history = PatientData().get_patient_service_history(patient_id)
            patient_df = patient_history["response_key"]
            lab_results = PatientData().get_patient_lab_result(patient_id)
            lab_df = lab_results["response_key"]

            Patient_data, Labresult_data, resultant_df = PatientData().get_min_max_daterange(patient_df, lab_df)
            patient_df_lab_record_join, resultant_df = PatientData().get_final_df_with_lab_results(Patient_data, resultant_df)
            # print(patient_df_lab_record_join)

            one_patient_record = PatientData().get_patient_derived_data(patient_df_lab_record_join, latest_app_date)
            # print(one_patient_record['Service_Date_max'])

            if one_patient_record.loc[0]["Patient_Practice"] != "Adhoc":
                self.patient_practice = "Regular"
                recommended_code = PatientData().recommendation(one_patient_record, rules_data)
            else:
                recommended_code = PatientData().allproceducecode(rules_data)

            print(recommended_code)
            end_time = time.time()
            self.patient_id = patient_id
            self.recommended_code = recommended_code
            status_msg = self.errorObj.SuccessMsg

            self.status = 200

        except Exception as e:
            end_time = time.time()
            self.logger.exception(str(e))
            self.status = 210
            status_msg = self.errorObj.FailureMsg

        response = self.responseObj.response_json_object(
            self.s_Name + str(self.errorObj.return_error_message(str(self.status))), end_time - start_time, self.status,
            status_msg, self.patient_practice, self.patient_id, self.recommended_code
        )
        return Response(response)
